In [1]:
class City:
    def __init__(self, n, x, y):
        self.name = n
        self.x = x
        self.y = y
        
    def __repr__(self):
        return 'City: ' + str(self.name) + ' ' + str(self.x) + ' ' + str(self.y) + '\n'

In [2]:
class Chromosome:
    def __init__(self, lst):
        self.val = lst
        self.fitness = -1
        self.cost = -1
    
    def set_fitness(self, cost):
        self.cost = cost
        self.fitness = 1.0/cost

    def __repr__(self):
        print(self.val)
        print(self.cost)
        return ''

In [3]:
import math
import random
import numpy as np
import pandas as pd
from copy import deepcopy
import matplotlib.pyplot as plt

In [4]:
def initial_pop(lst_cities, n, dist_mat):  
    # n -> population size
    # lst_cities -> list of objects
    generation = []
    for i in range(n):
        crom = lst_cities.copy()
        random.shuffle(crom)
        
        c = Chromosome(crom)
        f = cost(c, dist_mat)
        c.set_fitness(f)
        generation.append(c)
    return generation

In [5]:
def gen_dist_matrix(data):
    # data -> list of City objects
    n = len(data)
    mat = {} ##  Dic <first city, Dic <second city, distance> >
    for i,point in enumerate(data): # point -> city
        key = point.name
        mat[key] = {}
        for j in range(n):
            # distination city
            p2 = data[j]
            c = p2.name
            mat[key][c] = math.dist([point.x , point.y], [p2.x, p2.y])     
    return mat

In [6]:
def cost(crom, mat):
    # crom -> object of Chromosome
    total_cost = 0
    lst = crom.val
    for i, gene in enumerate(lst):
        if i == 0:
            continue
        total_cost += mat[gene.name][lst[i-1].name]
        
    total_cost += mat[lst[0].name][lst[-1].name]
    return total_cost

In [7]:
def Elitism(percent, old_pop):
    n = int(percent*len(old_pop))
    
    # choose most n fittest chromosomes
    sort = sorted(old_pop, key = lambda x: x.fitness, reverse=True)
    return sort[:n+1], sort[n+1:]

In [8]:
def tournment_selection(pop, n_candidates):
    candidates = random.choices(pop, k=n_candidates)
    return max(candidates, key = lambda x: x.fitness)
     

In [9]:
def partial_crossover(p1, p2, dist_mat):
    ln = len(p1.val)
    # random split range
    range1 = random.randint(0,ln-1)
    range2 = random.randint(0,ln-1)
    if range1 > range2:
        range1, range2 = range2, range1
        
    # children
    c1 = deepcopy(p1)
    c2 = deepcopy(p2)
    # for child1
    for gene in range(range1, range2+1):
        # find the duplicated city
        dup = p2.val[gene] # city
        dup_i = [d.name for d in c1.val].index(dup.name)
        c1.val[gene], c1.val[dup_i] = c1.val[dup_i], c1.val[gene]
        # swap between parent and child
        c1.val[gene] = p2.val[gene]
    
    # for child2
    for gene in range(range1, range2+1):
        # find the duplicated city
        dup = p1.val[gene] # city
        dup_i = [d.name for d in c2.val].index(dup.name)
        c2.val[gene], c2.val[dup_i] = c2.val[dup_i], c2.val[gene]
        # swap between parent and child
        c2.val[gene] = p1.val[gene]
    
    c1.set_fitness(cost(c1, dist_mat)) 
    c2.set_fitness(cost(c2, dist_mat)) 
    return c1, c2

In [10]:
def crossover_pop(prob, old_pop, n):
    # n -> number of chrom to be generated
    n_candidates = 5
    new_pop = []
    while len(new_pop) < n:
        # tournment selection
        parent1 = tournment_selection(old_pop, n_candidates)
        parent2 = tournment_selection(old_pop, n_candidates)
        
        # generate random prob
        r = random.random()
        if r < prob: # apply crossover
            c1, c2 = partial_crossover(parent1, parent2, dist_mat)
            if c1.fitness > parent1.fitness:
                new_pop.append(c1)
            else:
                new_pop.append(parent1)
            if c2.fitness > parent2.fitness:
                new_pop.append(c2)
            else:
                new_pop.append(parent2)
        else:  # pass the parents to the new pop
            new_pop.append(parent1)
            new_pop.append(parent2)
    # check if new_pop size == n
    if len(new_pop) != n:
        new_pop.pop()
    return new_pop

In [11]:
def mutation(prob, pop, dist_mat):
    new_pop = []
    for i,chrom in enumerate(pop):
        c = deepcopy(chrom)
        # generate random prob
        r = random.random()
        if r < prob: # apply mutation - swap 2 random genes
            ln = len(chrom.val)
            # random split range
            gene1 = random.randint(0,ln-1)
            gene2 = random.randint(0,ln-1)
            c.val[gene1], c.val[gene2] = c.val[gene2], c.val[gene1]
#         else:  # pass as it is
        c.set_fitness(cost(c, dist_mat)) 
        new_pop.append(c)
    return new_pop

In [ ]:
# # to take input from the GUI
# plt.plot(0,0)
# d = plt.ginput(10)
# data = [City(c,p[0],p[1]) for c,p in enumerate(d)]
# print(data)

In [12]:
raw = pd.read_csv('15-Points.csv')
raw

,x,y,City
0,5.500000e-08,9.860000e-09,1
1,-2.887330e+01,-7.980000e-08,2
2,-7.929160e+01,-2.140330e+01,3
3,-1.465770e+01,-4.338960e+01,4
4,-6.474730e+01,2.189820e+01,5
5,-2.905850e+01,-4.321670e+01,6
6,-7.207850e+01,1.815810e-01,7
7,-3.603660e+01,-2.161350e+01,8
8,-5.048080e+01,7.374470e+00,9
9,-5.058590e+01,-2.158820e+01,10


In [13]:
data = [City(c,x,y) for c,x,y in zip(raw['City'], raw['x'], raw['y'])]
print(data[2])

City: 3 -79.2916 -21.4033



In [25]:
n = len(data)
pop_n = 100

dist_mat = gen_dist_matrix(data)
gen = initial_pop(data, pop_n, dist_mat)  ## list of chromosomes


In [26]:
for i in range(100):
    # iteration
    new_pop = []
    elite, rest = Elitism(0.06, gen) 
    crossover_count = len(rest)
    crossed_over = crossover_pop(0.8, gen, crossover_count) #0.8
    mutated = mutation(0.3, crossed_over, dist_mat) #0.6
    new_pop.extend(elite)
    new_pop.extend(mutated)
    gen = new_pop.copy()


In [28]:
best = max(gen, key = lambda x: x.fitness)
best

[City: 5 -64.7473 21.8982
, City: 9 -50.4808 7.37447
, City: 15 -43.07 14.5548
, City: 2 -28.8733 -7.98e-08
, City: 13 -21.4983 7.31942
, City: 1 5.5e-08 9.86e-09
, City: 11 -0.135819 -28.7293
, City: 4 -14.6577 -43.3896
, City: 6 -29.0585 -43.2167
, City: 8 -36.0366 -21.6135
, City: 10 -50.5859 -21.5882
, City: 14 -57.5687 -43.2506
, City: 12 -65.0866 -36.0625
, City: 3 -79.2916 -21.4033
, City: 7 -72.0785 0.181581
]
284.3810904080332


In [17]:
def plot_sol(sol):
    x = []
    y = []
    for s in sol.val:
        x.append(s.x)
        y.append(s.y)
    plt.plot(x, y, marker = 'o')
    plt.show()

In [18]:
# plot_sol(best)

In [19]:
from matplotlib.animation import FuncAnimation
%matplotlib qt

In [29]:
x = []
y = []
for s in best.val:
    x.append(s.x)
    y.append(s.y)
x.append(best.val[0].x)
y.append(best.val[0].y)

fig, ax = plt.subplots()

def animate(i):
    ax.clear()
    ax.plot(x[:i+1], y[:i+1], marker = 'o')

    plt.ylim(-50, 30)
    plt.xlim(-90, 10)

ani = FuncAnimation(fig, animate, frames=20, interval=300, repeat=False)
plt.show()

In [ ]:
data2 = [City(c,p[0],p[1]) for c,p in enumerate(d)]
print(data2)

In [ ]:
data = data2